In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import web3

sys.path.append("..")
import web3_utils as w3ut

from brownie import *
import brownie

# Ganache

In [ ]:
!more surre_token1/contracts/surre_token_v1.sol

In [ ]:
brownie_project_dir = '/data/tutorial_brownie/surre_token1'
net = 'development'
w3 = w3ut.connect_to_brownie(brownie_project_dir, net)

In [ ]:
Token =  project.SurreToken1Project.Surrentum_v1
#Token =  project.Surrentum_v1
print(Token)
print(type(Token))
print(Token.deploy)

## Working with Accounts

In [ ]:
list(accounts)

In [ ]:
accounts[0]

In [ ]:
accounts[0].balance()

In [ ]:
w3ut.dir_(accounts[0])

In [ ]:
w3ut.print_balance(w3, accounts[0])
w3ut.print_balance(w3, accounts[1])

In [ ]:
os.environ["ETHERSCAN_TOKEN"]="4YRWQX6DB1TBBPTGRA4UU5CY2HY66M1QPE"
os.environ["WEB3_INFURA_PROJECT_ID"]="9153e25e52de479881b1798e0a91318f"

In [ ]:
w3ut.print_gas_price(w3)

In [ ]:
# From https://stackoverflow.com/questions/71341281/awaiting-transaction-in-the-mempool

if True:
    # In dev mode.
    #from brownie.network import gas_price
    #from brownie.network.gas.strategies import LinearScalingStrategy

    import brownie.network.gas.strategies
    gas_strategy = brownie.network.gas.strategies.LinearScalingStrategy("60 gwei", "70 gwei", 1.1)

    #if network.show_active() == "development":
    brownie.network.gas_price(gas_strategy)

## Deploy Token

In [ ]:
# Deploy the contract from account 1.
print(Token.deploy)
if True:
    token = Token.deploy({"from": accounts[1]})

In [ ]:
# Get the contract address.
surr_addr_ganache = token.address
print("surr_addr_ganache=", surr_addr_ganache)

In [ ]:
# Print the balance of the SURRE token.
w3ut.print_balance(w3, accounts[0], token=token)
w3ut.print_balance(w3, accounts[1], token=token)

In [ ]:
token

In [ ]:
# Print the signature.
print(token.transfer)

# NatSpec.
token.transfer.info()

## Transactions

In [ ]:
# Transfer SURRE token from account 1 to account 0.
tx = token.transfer(accounts[0], "1 ether", {'from': accounts[1]})

In [ ]:
# Print Python interfact of transaction object.
print(w3ut.dir_(tx))

In [ ]:
print(tx)
print("status=\n%s" % tx.status)
print("tx.info=", tx.info())

In [ ]:
# Print the balance of the SURRE token.
w3ut.print_balance(w3, accounts[0], token=token)
w3ut.print_balance(w3, accounts[1], token=token)

In [ ]:
token.transfer.call(accounts[0], "1 ether", {'from': accounts[1]})

In [ ]:
# Print the balance of the SURRE token.
w3ut.print_balance(w3, accounts[0], token=token)
w3ut.print_balance(w3, accounts[1], token=token)

In [ ]:
# Non-blocking transactions.
transactions = [
    accounts[0].transfer(accounts[i], "1 ether", required_confs=0)
    for i in range(1, 4)
]
[tx.status for tx in transactions]

In [ ]:
[tx.status for tx in transactions]

In [ ]:
for i in range(0, 4):
    w3ut.print_balance(w3, accounts[i], token=token)

## Accessing transaction data

In [ ]:
# transactions broadcasted during the current session.
list(history)

In [ ]:
#dir(history[0])

tx_id = str(history[0].txid)
print(tx_id)
tx = chain.get_transaction(tx_id)
print(tx.info())

## Inspecting and debugging transactions

In [ ]:
tx = Token[0].transfer(accounts[1], 100e18, {'from': accounts[1], "required_confs": 0})

In [ ]:
tx.status

In [ ]:
#tx.revert_msg

In [ ]:
#tx.info()

In [ ]:
tx.events

In [ ]:
#tx.traceback()

# Networks and deployment

In [ ]:
net = 'goerli'
w3 = w3ut.connect_to_brownie(brownie_project_dir, net)

In [ ]:
list(accounts)

In [ ]:
# cp /data/tutorial_brownie/proj_files/goerli.json /root/.brownie/accounts/goerli.json
accounts.load('goerli')

account = accounts[0]
print(account)

## Connect to blockchain

In [ ]:
!curl --url https://goerli.infura.io/v3/$WEB3_INFURA_PROJECT_ID -X POST -H "Content-Type: application/json" -d '{"jsonrpc":"2.0","method":"eth_blockNumber","params":[],"id":1}'

In [ ]:
# Test Infura key.
import os
import web3

infura_key = os.environ["WEB3_INFURA_PROJECT_ID"]

infura_url = f"https://goerli.infura.io/v3/{infura_key}"
w3 = web3.Web3(web3.Web3.HTTPProvider(infura_url))
print("isConnected=", w3.isConnected())

In [ ]:
w3ut.print_balance(w3, account)

## Deploy token

In [ ]:
if False:
    # Deploy Token contract from one account.
    token = Token.deploy({"from": account}, publish_source=True)

In [ ]:
surr_addr = "0xBD34d0472A783D0849D981052327e6c6544E90C5"
#surr_addr = "0x2fFF61848990F4C54373476B90b1C1268EF8c229"
surr_tx = "0x62f8c551f962322e1af638768ad116bd6a3efed0f795ae4d606e09e35636f713"
w3ut.print_tx(surr_tx)

In [ ]:
abi_file = f"{brownie_project_dir}/build/contracts/Surrentum_v1.json"
import json
f = open(abi_file)
abi = json.load(f)["abi"]

In [ ]:
print(surr_addr)
contract = w3.eth.contract(address=surr_addr, abi=abi)

In [ ]:
w3ut.print_balance(w3, account, token=contract)

In [ ]:
# Print the interface of the Eth contract.
contract.functions.__dict__.keys()

In [ ]:
print("name=", contract.functions.name().call())
print("symbol=", contract.functions.symbol().call())
totalSupply = contract.functions.totalSupply().call()
print("totalSupply=", w3.fromWei(totalSupply, 'ether'))

# Send SURR

In [ ]:
src_account = str(account)
print("src_account=", src_account)
w3ut.print_balance(w3, src_account, token=contract)
w3ut.print_balance(w3, src_account)

In [ ]:
#w3ut.dir_(w3.eth.account)

In [ ]:
dst_account = "0xA8785E587809ef7217956d88ec4266FeAfb8DFcA"
# Grisha
#dst_account = "0x855Da94Db7d3A843e53b7728958A49423543F871"
print("dst_account=", dst_account)

w3ut.print_balance(w3, dst_account, token=contract)
w3ut.print_balance(w3, dst_account)

In [ ]:
# This doesn't work on Infura.
if True:
    eth = w3.toWei(0.01, "ether")
    w3.eth.default_account = src_account
    #contract.functions.transfer(dst_account, eth).transact({'from': src_account})
    contract.functions.transfer(dst_account, eth).transact()

In [ ]:
print(w3.eth.gas_price)

In [ ]:
#priv_key1 = "0x4f3edf983ac636a65a842ce7c78d9aa706d3b113bce9c46f30d7d21715b23b1d"

# Send Eth from account1 to 2.
nonce = w3.eth.getTransactionCount(src_account)
print("nonce=", nonce)

eth = w3.toWei(0.02, "ether")
tx = contract.functions.transfer(
    dst_account,
    eth,
).build_transaction({
    #'chainId': 1,
    #'gas': 70000,
    #'maxFeePerGas': w3.toWei('100', 'gwei'),
    #'maxPriorityFeePerGas': w3.toWei('1', 'gwei'),
    "gasPrice": w3.eth.gas_price,
    "gas": 100000,
    'nonce': nonce,
})
print("tx=", tx)

signed_tx = w3.eth.account.signTransaction(tx, account.private_key)
print("signed_tx=", signed_tx)

tx_hash = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
print("tx_hash=", tx_hash)

In [ ]:
#dir(tx)

In [ ]:
#w3ut.print_tx(tx)
print(tx_hash)
tx = chain.get_transaction(tx_hash)
print(tx.info())
print(tx.status)

# Uniswap

In [ ]:
#!pip install uniswap-python

In [ ]:
from uniswap import Uniswap

# "YOUR ADDRESS"          # or None if you're not going to make transactions
address = src_account
# "YOUR PRIVATE KEY"  # or None if you're not going to make transactions
private_key = account.private_key
# specify which version of Uniswap to use
version = 3
# "WEB3 PROVIDER URL"    # can also be set through the environment variable `PROVIDER`
provider = infura_url
uniswap = Uniswap(address=address, private_key=private_key, version=version, provider=provider)

# Some token addresses we'll be using later in this guide
eth = "0x0000000000000000000000000000000000000000"
#bat = "0x0D8775F648430679A709E98d2b0Cb6250d2887EF"
bat = "0x70cba46d2e933030e2f274ae58c951c800548aef"  # On Goerli.
#dai = "0x6B175474E89094C44Da98b954EedeAC495271d0F"
dai = "0xdc31ee1784292379fbb2964b3b9c4124d8f89c60"  # On Goerli

In [ ]:
#uniswap.get_price_input(eth, dai, 10**18)

In [ ]:
#uniswap.get_price_output(eth, dai, 1_000 * 10**18)

In [ ]:
#uniswap.get_token(eth)
#uniswap.get_token(dai)
uniswap.get_token(bat)

In [ ]:
web3_lib.toChecksumAddress

In [ ]:
#uniswap.get_pool_instance(dai, bat)
uniswap.get_pool_instance(web3_lib.Web3.toChecksumAddress(dai), web3_lib.Web3.toChecksumAddress(bat))

In [ ]:
#uniswap.make_trade(eth, dai, 1*10**15)  # sell 1 ETH for BAT
uniswap.make_trade(
    web3_lib.Web3.toChecksumAddress(eth), web3_lib.Web3.toChecksumAddress(bat),
    1*10**15)  # sell 1 ETH for BAT

In [ ]:
# Get the balance of ETH in an exchange contract.
#uniswap.get_ex_eth_balance(bat)

In [ ]:
# Get balance of ETH for you address (in wei).
uniswap.get_eth_balance()

In [ ]:
# Get the balance of a token for your address.
uniswap.get_token_balance(surr_addr)

In [ ]:
uniswap.get_token_balance(bat)

In [ ]:
#uniswap.mint_liquidity()

In [ ]:
# Returns the amount of DAI you get for 1 ETH (10^18 wei)
#uniswap.get_price_input(eth, dai, 10**18)

In [ ]:
# ContractLogicError: execution reverted
#uniswap.create_pool_instance(eth, surr_addr)

In [ ]:
# uniswap_Factory
factory_abi = json.loads('[{"inputs":[{"internalType":"address","name":"_feeToSetter","type":"address"}],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"token0","type":"address"},{"indexed":true,"internalType":"address","name":"token1","type":"address"},{"indexed":false,"internalType":"address","name":"pair","type":"address"},{"indexed":false,"internalType":"uint256","name":"","type":"uint256"}],"name":"PairCreated","type":"event"},{"constant":true,"inputs":[{"internalType":"uint256","name":"","type":"uint256"}],"name":"allPairs","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"allPairsLength","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"tokenA","type":"address"},{"internalType":"address","name":"tokenB","type":"address"}],"name":"createPair","outputs":[{"internalType":"address","name":"pair","type":"address"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"feeTo","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"feeToSetter","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"address","name":"","type":"address"}],"name":"getPair","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"_feeTo","type":"address"}],"name":"setFeeTo","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"_feeToSetter","type":"address"}],"name":"setFeeToSetter","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}]')
#print(factory_abi)
factory_address = '0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f'
factory_contract = web3.eth.contract(address=factory_address, abi=factory_abi)

#returns a count of all the trading pairs on uniswap
allPairsLength = factory_contract.functions.allPairsLength().call()
print(allPairsLength)

# get the uniswap pair address for pricing

In [ ]:
# Send Eth from account1 to 2.
nonce = web3.eth.getTransactionCount(src_account)
print("nonce=", nonce)

# https://docs.uniswap.org/contracts/v2/reference/smart-contracts/factory
#eth_addr = "0x0000000000000000000000000000000000000000"
#func = factory_contract.functions.createPair(eth_addr, surr_addr)
bat_addr = "0x0D8775F648430679A709E98d2b0Cb6250d2887EF"
func = factory_contract.functions.createPair(bat_addr, surr_addr)
print(func)

tx = func.build_transaction({
     #'chainId': 1,
     #'maxFeePerGas': web3.toWei('1000', 'gwei'),
     #'maxPriorityFeePerGas': web3.toWei('1', 'gwei'),
     "gasPrice": web3.eth.generate_gas_price(),
     "gas": 10000000,
     'nonce': nonce,
})
print("tx=", tx)

#gas_estimate = web3.eth.estimateGas(tx)
#print("gas_estimate", gas_estimate)

signed_tx = web3.eth.account.signTransaction(tx, account.private_key)
print("signed_tx=", signed_tx)

tx_hash = web3.eth.sendRawTransaction(signed_tx.rawTransaction)
print("tx_hash=", tx_hash)
#print("tx_status=", tx.status)

In [ ]:
tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)
import pprint
pprint.pprint(tx_receipt)
#print(tx_receipt.contractAddress)

In [ ]:
tx_receipt

In [ ]:
tx_receipt.status

In [ ]:
print(tx_hash)
tx = chain.get_transaction(tx_hash)
print(tx.info())

In [ ]:
factory_contract.events.PairCreated.processReceipt(tx_receipt)

## Swap

In [ ]:
#uniswap_addr = "0xCCa773F0FeC05569f18381D8bDf4cb684Aa1d7E5"
uniswap_addr = "0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f"

uniswap = Uniswap(address=address, private_key=private_key, version=version, provider=provider)
                 #factory_contract_addr=uniswap_addr)

In [ ]:
uniswap.get_price_input(eth, dai, 10**18)

In [ ]:
#uniswap.
allPairsLength = uniswap.allPairsLength().call()
print(allPairsLength)

# Etherscan-python

https://github.com/pcko1/etherscan-python

In [ ]:
#!pip install etherscan-python

In [ ]:
#import etherscan
from etherscan import Etherscan
#eth = Etherscan(os.environ["ETHERSCAN_TOKEN"], net="goerli")
eth = Etherscan(os.environ["ETHERSCAN_TOKEN"])

In [ ]:
factory_address = '0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f'
abi = eth.get_contract_abi(factory_address)

factory_abi = json.loads(abi)
print(factory_abi)
factory_contract = web3.eth.contract(address=factory_address, abi=factory_abi)

In [ ]:
#dir(factory_contract)
factory_contract.functions.createPair

In [ ]:
# There are several ways to interact with a contract / service
# - Use REST API of the service
# - Use SDK (e.g., JS or Python): python-uniswap
# - Interact with contract directly on Ethereum

In [ ]:
eth.get_eth_last_price()